In [17]:
from google.cloud import storage
from pandas_ods_reader import read_ods
from io import BytesIO
import pandas as pd

In [5]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('opengeokpi-bucket')


# load equipements counting

In [19]:
# get data storage
blob = bucket.blob('raw/equipements/bpe18_ensemble.csv')
content =blob.download_as_string()

In [20]:
# read equipement counting data
bpe18_ensemble = pd.read_csv(BytesIO(content), sep = ';',dtype=str)

In [21]:
bpe18_ensemble.head()

,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,NB_EQUIP
0,84,01,01001,01001,2018,A401,2
1,84,01,01001,01001,2018,A404,4
2,84,01,01001,01001,2018,A504,1
3,84,01,01001,01001,2018,A507,1
4,84,01,01001,01001,2018,B203,1


# load equipements codes and types

In [ ]:
# get data storage
blob = bucket.blob('raw/equipements/BPE_gammes_2018_internet.ods')
content =blob.download_as_string()

In [12]:
# read ods file
sheet_idx = 1
BPE_gammes_2018 = read_ods(BytesIO(content), sheet_idx)

In [13]:
# drop the first 4 rows
BPE_gammes_2018 = BPE_gammes_2018.drop([0, 1,2,3])

In [15]:
# rename columns
BPE_gammes_2018.columns = ['CodeEquipement', 'regroup1', 'regroup2','regroup3','EquipementLabel','EquipementGamme','EquipementDomainCode','EquipementDomainLabel']

In [24]:
BPE_gammes_2018.EquipementDomainLabel.unique()

array(['Services aux particuliers', 'Commerces', 'Enseignement', 'Santé',
       'Transports et déplacements', 'Sports, loisirs et culture'],
      dtype=object)

In [16]:
display(BPE_gammes_2018)

,CodeEquipement,regroup1,regroup2,regroup3,EquipementLabel,EquipementGamme,EquipementDomainCode,EquipementDomainLabel
4,AR03,A206,A207,A208,"Bureau de poste, relais poste, agence postale",proximité,A,Services aux particuliers
5,A301,A301,None,None,Réparation automobile et de matériel agricole,proximité,A,Services aux particuliers
6,A401,A401,None,None,Maçon,proximité,A,Services aux particuliers
7,A402,A402,None,None,"Plâtrier, peintre",proximité,A,Services aux particuliers
8,A403,A403,None,None,"Menuisier, charpentier, serrurier",proximité,A,Services aux particuliers
...,...,...,...,...,...,...,...,...
109,E01G,E107,E108,E109,Gare,supérieure,E,Transports et déplacements
110,F109,F109,None,None,Parcours sportif/santé,supérieure,F,"Sports, loisirs et culture"
111,F306,F306,None,None,"Théâtre, art de rue, cirque",supérieure,F,"Sports, loisirs et culture"
112,F303,F303,None,None,Cinéma,supérieure,F,"Sports, loisirs et culture"


In [ ]:
# Merge equipement counting with ewuipement labels

In [22]:
bpe18_ensemble_label = pd.merge(left = bpe18_ensemble, 
                                right = BPE_gammes_2018,
                                how='left',
                                left_on='TYPEQU', 
                                right_on='CodeEquipement')

In [23]:
bpe18_ensemble_label.head()

,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,NB_EQUIP,CodeEquipement,regroup1,regroup2,regroup3,EquipementLabel,EquipementGamme,EquipementDomainCode,EquipementDomainLabel
0,84,01,01001,01001,2018,A401,2,A401,A401,None,None,Maçon,proximité,A,Services aux particuliers
1,84,01,01001,01001,2018,A404,4,A404,A404,None,None,"Plombier, couvreur, chauffagiste",proximité,A,Services aux particuliers
2,84,01,01001,01001,2018,A504,1,A504,A504,None,None,"Restaurant, restaurant rapide",proximité,A,Services aux particuliers
3,84,01,01001,01001,2018,A507,1,A507,A507,None,None,Institut de beauté – onglerie,proximité,A,Services aux particuliers
4,84,01,01001,01001,2018,B203,1,B203,B203,None,None,Boulangerie,proximité,B,Commerces
